In [1]:
import pygame
import time
import random
import neat
import os
import cv2
import tensorflow as tf
import numpy as np
pygame.font.init()  # init font

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from tensorflow.keras.models import load_model

#### Load model

In [3]:
fist_tracker = load_model('fist_tracker_aug4_1.h5')

#### Creating game

In [4]:
# Initial inputs for our game 
X_screen = 500                                    # width of screen
Y_screen = 800                                    # length of screen
bird_image = pygame.image.load("bird.png")        # image of our object bird
STAT_FONT = pygame.font.SysFont("comicsans", 50)  # font of score drawn on screen

In [5]:
# Initial position of bird
bird_X = X_screen * 0.15   # initial x position of bird which is constant during game
bird_Y = Y_screen * 0.4    # initial y position of bird

In [6]:
# Assumptions for obstacles
rectangle_width = 70      # width of our obstacle
rectangle_gap = 350       # gap between top and bottom obstacle

In [7]:
class Bird:
    """
    Class Bird is representation of our bird in game. 
    It has functions up() which is used to move bird up. 
    Function change_position changing position of bird up or down, depends on player input. 
    Next function draw() draw bird on display and also maintain limitation of position of our bird.
    """
    
    def __init__(self, bird_X, bird_Y):
        self.x = bird_X          # x position of bird
        self.y = bird_Y          # y position of bird
        self.move = -7           # constant move of bird. It is negative because it is an upward move
        self.image = bird_image  # image of bird
        self.repeat = 0          # it shows how long program works without user action
        self.length = bird_image.get_height()
        self.tick_count = 0
        self.vel = 0
        self.height = self.y
        
    def up(self):
        """
        make the bird jump
        :return: None
        """
        self.vel = -10.5
        self.tick_count = 0
        self.height = self.y
    
    def change_position(self):
        """
        make the bird move
        :return: None
        """
        self.tick_count += 1

        # for downward acceleration
        displacement = self.vel*(self.tick_count) + 0.5*(3)*(self.tick_count)**2  # calculate displacement

        # terminal velocity
        if displacement >= 16:
            displacement = (displacement/abs(displacement)) * 16

        if displacement < 0:
            displacement -= 2

        self.y = self.y + displacement

    
    
    def collision(self):                                        
        end = False
        if self.y < 0:          # checking if bird touch the top or bottom of screen, if so game ends
            self.y = 0
        if self.y > (Y_screen - self.length):
            self.y = Y_screen - self.length
        return end
            
    
    def draw(self, screen):
        self.image = bird_image
        screen.blit(self.image, (self.x, self.y))                 # drawing bird based on new y position

In [8]:
class Obstacle:
    """
    Next class Obstacle generate obstacles in our game. There are four functions:
    First one set_high() generate random height of obstacle in range between 200 and 450. 
    The second function: change_position() is responsible for move of pipes. 
    it also checking if obstacle is out of window. If yes it creating new obstacle with new random height.
    Next function draw() creating a rectangles which are displayed on screen. 
    The last function collide() is checking is there a collision between bird and created obstacles. if yes it returns False. 
    """
    
    def __init__(self, x):
        self.x = x                       # x position of obstacle. By default size of screen (500)
        self.height = 0                  # height of top obstacle
        self.score = 0                   # how many obstacles player passed
        self.width = rectangle_width     # width of obstacle
        self.gap = rectangle_gap         # gap between top and bottom obstacle
        self.move = -3                   # speed of obstacle
        self.Y_screen = Y_screen         # size of screen 
        self.set_height()                # setting the initial value of height
        
    def set_height(self):
        self.height = random.randrange(200, 450)     # random value of height between 200 and 450 
    
    def change_position(self):                       
        self.x += self.move                          # changing position of obstacle based on initial value of speed
        if self.x <= -self.width:                    # if obstacle miss the screen creating new obstacle at the beggining
            self.x = 500
            self.set_height()
            self.score += 1
        return self.score
            
    def draw(self, screen):
        pygame.draw.rect(screen, (0, 0, 0), (self.x, 0, self.width, self.height))                        # drawing top obstacle
        height_second = self.Y_screen - self.height - self.gap       # height of the second obstacle based on length of screen, gap and height of first one
        pygame.draw.rect(screen, (0, 0, 0), (self.x, self.height + self.gap, self.width, height_second)) # drawing bottom obstacle
        
    def collide(self, bird):
        end = False
        if bird.x in range(self.x - self.width, self.x):
            if bird.y <= self.height or bird.y >= (self.height + self.gap - 50):   # checking if bird is on the same level as obstacles
                end = True
        return end

In [9]:
def draw_screen(screen, bird, obst, score2):
    """
    Function used in the main fitness function. 
    It is used to draw all the necessary elements and refresh the screen.
    """
    
    screen.fill((255, 255, 255))      # filling background with white colour
    obst.draw(screen)                 # drawing obstacles

    bird.draw(screen)
    
    score_label = STAT_FONT.render("Score: " + str(score2),1,(0,255,0))      # defining text of score
    screen.blit(score_label, (X_screen - score_label.get_width() - 15, 10))  # drawing score on board
    pygame.display.update()           # displaying screen

In [10]:
def game():
    cap = cv2.VideoCapture(0)                                 # Open video capture
    screen = pygame.display.set_mode((X_screen, Y_screen))    # Set up the game screen

    
    bird = Bird(bird_X, bird_Y)                               # Create a bird object
    obst = Obstacle(X_screen)                                 # Create an obstacle object
    
    clock = pygame.time.Clock()                               # Create a clock object
    score = 0                                                 # Initialize the score
    start = False                                             # Flag to indicate if the game has started
    
    while True:   # Main game loop                
        for event in pygame.event.get():
            if event.type == pygame.MOUSEBUTTONUP:            # Start the game if the mouse button is clicked
                start = True
                break
            if event.type == pygame.QUIT:                     # Quit the game if the window is closed
                pygame.quit()
                return
        if start:
            break
            
    while start and cap.isOpened():
        _ , frame = cap.read()                                     # Read a frame from the video capture
        frame = frame[50:500, 50:500,:]                            # Crop the frame
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)               # Convert frame to RGB
        resized = tf.image.resize(rgb, (120,120))                  # Resize the frame
        yhat = fist_tracker.predict(np.expand_dims(resized/255,0)) # Predict with the fist tracker
        cv2.imshow('Fist', frame)                                  # Show the frame with fist detection
        
        if yhat[0] > 0.5:
            y = 5
        else:
            y = 2
    
        for x in range(y):

            clock.tick(20)
            for event in pygame.event.get():     # checking if user click the quit button 
                if event.type == pygame.QUIT:
                    start = False
                    pygame.quit()

            bird.change_position()               # Change the position of the bird
            bird.collision()                     # Check for collision with the bird
            score = obst.change_position()       # Change the position of the obstacle
            
            
            if yhat[0] > 0.5:
                bird.up()                        # Make the bird go up if fist is detected
                print("fist")

            if obst.collide(bird): start = False      # Quit the game if collision occurs
            draw_screen(screen, bird, obst, score)    # Draw the game screen
    
    pygame.quit()                                # Quit pygame
    cap.release()                                # Release the video capture
    cv2.destroyAllWindows()                      # Close all windows

In [11]:
game()

1/1 [==============================] - 0s 481ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 130ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 146ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 144ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 155ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 132ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 146ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 148ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 148ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 122ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 155ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 127ms/step
fist
fist
fist
fist
fist
1/1 [==============================] - 0s 133ms/step
fist
fist
f

In [12]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))


    if yhat[0] > 0.5: 
        # Controls the main rectangle
        print("fist")
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

NameError: name 'facetracker' is not defined